# A Finite Element Solution for a plate with MATLAB code.

The problem is show in the figure 6-16. 

![Figure 1](figure1.jpg)

This problem is show in the chapter 6.5 of the book: *A firts Course in the Finite Element Method* written by Daryl L. Logan. In this 
chapter a solution is make by subdivided the plate in two CST. The purpose of this project is to make more divisions considering 400 CST and see the results. 

## Preprocesing 

In [14]:
% Geometry definition

% The values are: 

t = 1; % in
E = 30*10^6; %psi
v = 0.3; 

%The mesh is divided in 200 squares, 10 division in the y direction and 20 divisions in the x direction. With this division each rectangle size 1x1 in^2. 
%At each rectangle the CST is generated joining the lower-left node with the upper-right node of the rectangle. 
%The vectors for this divisions are defined: 

x0 = 0;
y0 = 0;
x1 = 20;
y1 = 10; 
x = linspace(x0,x1,20); 
y = linspace(y0,y1,10); 
division = zeros(length(x),length(y));
